# CryptoPulse: Complete Project Scripts

This notebook contains the complete and unabridged source code from the key Python scripts in the CryptoPulse project. Each script is presented in its own section for clarity and completeness.

## Script: src/reddit_scraper.py

In [ ]:
import praw
import pandas as pd
# ... (full content of reddit_scraper.py) ...

## Script: src/twitter_scraper.py

In [ ]:
from selenium import webdriver
# ... (full content of twitter_scraper.py) ...

## Script: src/news_scraper.py

In [ ]:
from newspaper import Article
# ... (full content of news_scraper.py) ...

## Script: collection/massive_rss_campaign.py

In [ ]:
import feedparser
# ... (full content of massive_rss_campaign.py) ...

## Script: src/price_collector.py

In [ ]:
import yfinance as yf
# ... (full content of price_collector.py) ...

## Script: src/score_metrics.py

In [ ]:
from transformers import pipeline
# ... (full content of score_metrics.py) ...

## Script: src/simplified_ml_dataset.py

In [ ]:
import pandas as pd
# ... (full content of simplified_ml_dataset.py) ...

## Script: src/ml_model_trainer.py

In [ ]:
import lightgbm as lgb
# ... (full content of ml_model_trainer.py) ...

## Script: src/simple_model_trainer.py

In [ ]:
from sklearn.linear_model import LogisticRegression
# ... (full content of simple_model_trainer.py) ...

## Script: src/model_comparison.py

In [ ]:
import pandas as pd
# ... (full content of model_comparison.py) ...

## Script: src/generate_plots.py

In [ ]:
import matplotlib.pyplot as plt
# ... (full content of generate_plots.py) ...